In [ ]:
%pip install pymongo tqdm

In [ ]:
import pandas as pd
import json
from collections import Counter
from tqdm import tqdm
from pymongo import MongoClient
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
conf = pyspark.SparkConf()


import copy
import numpy as np


tqdm.pandas()

MONGO_CONN = 'mongodb+srv://<username>:<password>@retail-demo.2wqno.mongodb.net/?retryWrites=true&w=majority'

In [ ]:
# from pymongo import MongoClient

# client = MongoClient(MONGO_CONN)
# db = client["search"]
# db['catalog_n'].delete_many({})
# pid = pd.DataFrame.from_records(db['catalog_n'].find({},{"_id": 0, "product_uid": 1}))
# atp = pd.DataFrame.from_records(db['atp_status'].find({},{"_id":0}))
# df = pid.merge(atp, on='product_uid', how="left")
# db['atp_status'].delete_many({})
# db['atp_status'].insert_many(df.to_dict(orient='records'))

In [ ]:
atp = spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "atp_status_myn").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

atp = atp.drop("_id")
atp.writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.operationType', "update").\
            option('spark.mongodb.upsertDocument', True).\
            option('spark.mongodb.idFieldList', "id").\
            option("forceDeleteTempCheckpointLocation", "true").\
            option("checkpointLocation", "/tmp/retail-atp-myn4/_checkpoint/").\
            outputMode("append").\
            start()

In [ ]:
images=spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "mynimages").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option('spark.mongodb.aggregation.pipeline',[{"$project":{'product_id':0}}]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

images = images.drop("_id")
images = images.drop("filename")
images.writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option("checkpointLocation", "/tmp/retail-image-myn3/_checkpoint/").\
            outputMode("append").\
            start()

In [ ]:
prd_desc=spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "prd_desc_myn").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

prd_desc = prd_desc.drop("_id")
prd_desc = prd_desc.withColumn("title", F.col("productDisplayName"))
prd_desc = prd_desc.withColumn("mfg_brand_name", F.col("brandName"))

prd_desc = prd_desc.withColumn("id", F.col("id").cast(T.StringType()))
prd_desc.writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option("checkpointLocation", "/tmp/retail-prd-desc-myn3/_checkpoint/").\
            outputMode("append").\
            start()

In [ ]:
prd_score=spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "prd_score_myn").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

prd_score = prd_score.drop("_id")
prd_score.select("id", "score").writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            option("checkpointLocation", "/tmp/retail-prd-score-myn1/_checkpoint/").\
            option("forceDeleteTempCheckpointLocation", "true").\
            outputMode("append").\
            start()

In [ ]:
price = spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "price_myn").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option("forceDeleteTempCheckpointLocation", "true").\
            option('spark.mongodb.aggregation.pipeline',[{"$project":{"_id":0}}]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

price = price.drop("_id")
price.writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            option("checkpointLocation", "/tmp/retail-price-myn3/_checkpoint/").\
            option("forceDeleteTempCheckpointLocation", "true").\
            outputMode("append").\
            start()